In [ ]:
# we're installing from source since the model we will be using is brand new at the time of writing
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install -q datasets

In [ ]:
import os
import cv2
import random
import torch
import numpy as np

from torch.utils.data import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image


##data preprocessing

In [ ]:
#splite the training and testing path

train_size = 0.9
paths = []
path_to_subset = '/content/drive/MyDrive/Work_space/Data set/Cyber/function4/Cyber/train'
for folder in os.listdir(path_to_subset):

    for image in os.listdir(os.path.join(path_to_subset, folder)):
        path_to_image = os.path.join(path_to_subset, folder, image)

        #check image none or not none
        img = cv2.imread(path_to_image)
        if img is not None:
          paths.append( path_to_image)

paths = sorted(paths)
paths[0]
random.shuffle(paths)
assert 0.0 < train_size <= 1.0
thresh = round(len(paths) * train_size)
train_paths = paths[:thresh]
test_paths = paths[thresh:]
print(train_paths[0], test_paths[1])

In [ ]:
print(train_paths[0:5])
print(len(train_paths))
print(len(test_paths))

In [ ]:
#Label and classes extraction
def labels(root):
  classes=[]
  lab=[]
  labels=[]
  for idx in range(len(root)):
    # get image
    image_path =root[idx]

    # get labels
    label = image_path.split(os.path.sep)[-2].split(' ')
    lab.append(label)

  mlb = MultiLabelBinarizer()
  labels = mlb.fit_transform(lab)
  classes = mlb.classes_

  return labels,classes

In [ ]:
#Show label and classes
m_labels,classes=labels(train_paths)
print(m_labels)
print(classes)

In [ ]:
#Id to label and label to id for fine tune the model
id2label = {id: label for id, label in enumerate(classes)}
label2id = {v: k for k, v in id2label.items()}
print(id2label)
print(label2id)

In [ ]:
id2label[2]

##Create PyTorch Dataset

In [ ]:
class MultiLabelDataset(Dataset):
  def __init__(self, root, transform,labels):
    self.root = root
    self.transform = transform
    self.labels=labels

  def __getitem__(self, idx):
    # get image
    image_path = self.root[idx]
    image = Image.open(image_path).convert("RGB")

    # prepare image for the model
    pixel_values = self.transform(image)

    # get labels
    label = self.labels[idx].astype(np.float32)

    # turn into PyTorch tensor
    label = torch.from_numpy(label)

    return pixel_values,label


  def __len__(self):
    return len(self.root)

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

#select the pretraining model
model_id = "google/vit-base-patch16-224-in21k" # "google/siglip-so400m-patch14-384"

processor = AutoImageProcessor.from_pretrained(model_id)
model = AutoModelForImageClassification.from_pretrained(model_id, problem_type="multi_label_classification", id2label=id2label,label2id=label2id)

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
#show processing details in pre training model
processor

In [ ]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

# get appropriate size, mean and std based on the image processor
size = processor.size["height"]
mean = processor.image_mean
std = processor.image_std

transform = Compose([
    Resize((size, size)),
    ToTensor(),
    Normalize(mean=mean, std=std),
])

#add the dataset to pytorch dataset
train_dataset = MultiLabelDataset(root=train_paths, transform=transform,labels=m_labels)


##Get the sample

In [ ]:
pixel_values, labels = train_dataset[12]
print(pixel_values.shape)

In [ ]:
unnormalized_image = (pixel_values.numpy() * np.array(std)[:, None, None]) + np.array(mean)[:, None, None]
unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
Image.fromarray(unnormalized_image)

In [ ]:
labels

In [ ]:
s=torch.nonzero(labels).squeeze().tolist()
print(s)

In [ ]:
[id2label[s]]

##Create PyTorch DataLoader

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    data = torch.stack([item[0] for item in batch])
    target = torch.stack([item[1] for item in batch])
    return data, target

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=32, shuffle=True)


In [ ]:
batch = next(iter(train_dataloader))
print(batch)

In [ ]:
print(batch[0].shape)
print(batch[1].shape)

Verify initial loss

In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


outputs = model(pixel_values=batch[0].to(device), labels=batch[1].to(device))

In [ ]:
outputs.loss

##Training the model

In [ ]:
# handy utility I found at https://github.com/wenwei202/pytorch-examples/blob/ecbb7beb0fac13133c0b09ef980caf002969d315/imagenet/main.py#L296
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
from torch.optim import AdamW
from tqdm.auto import tqdm

# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

losses = AverageMeter()

model.train()
for epoch in range(10):  # loop over the dataset multiple times
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values, labels = batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(
            pixel_values=pixel_values.to(device),
            labels=labels.to(device),
        )

        # calculate gradients
        loss = outputs.loss
        losses.update(loss.item(), pixel_values.size(0))
        loss.backward()

        # optimization step
        optimizer.step()

        if idx % 100 == 0:
            print('Epoch: [{0}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                   epoch, loss=losses,))

save model

In [ ]:
import torch

# Assuming 'model' is your PyTorch model
# Save the entire model
torch.save(model, 'fine_tuned_vit-base-patch16-224-in21k.pth')

# Or save only the model state dictionary
torch.save(model.state_dict(), 'fine_tuned_vit-base-patch16-224-in21k_state_dict.pth')

load save model

In [ ]:
import torch

# Load the entire model
model2 = torch.load('/content/fine_tuned_vit-base-patch16-224-in21k.pth')

# Move the model to the appropriate device if necessary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2.to(device)

##Inference

In [ ]:
# load image to test on
image = Image.open(test_paths[47])
image

In [ ]:
model2.eval()

# prepare image for the model
pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

# forward pass
with torch.no_grad():
  outputs = model2(pixel_values)
  logits = outputs.logits


In [ ]:

# turn into probabilities by applying sigmoid
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
#print(probs)
# select the probabilities > a certain threshold (e.g. 50%) as predicted
predictions = np.zeros(probs.shape)
#print(predictions)
predictions[np.where(probs >= 0.5)] = 1 # turn predicted id's into actual label names
#print(predictions)
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

##Evaluation

In [ ]:

def pred_pre( root,processor):
  root = root
  processor = processor
  true_labels=[]
  predicted_labels=[]
  labletoid={'Action': 0, 'Drugs': 1, 'Gambling': 2, 'LSD': 3, 'Weapon': 4}


  for idx in range(len(root)):
    # get image
    image_path = root[idx]
    image = cv2.imread(image_path)
    #print(idx)
    try:
      label = image_path.split(os.path.sep)[-2].split(' ')
      #print(image_path)
      # get labels
      labels2=labletoid[label[0]]

      image = Image.open(image_path)

      # prepare image for the model
      pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

      # forward pass
      with torch.no_grad():
        outputs = model2(pixel_values)
        logits = outputs.logits


      # turn into probabilities by applying sigmoid
      sigmoid = torch.nn.Sigmoid()
      probs = sigmoid(logits.squeeze().cpu())

      # select the probabilities > a certain threshold (e.g. 50%) as predicted
      predictions = np.zeros(probs.shape)

      predictions[np.where(probs >= 0.5)] = 1 # turn predicted id's into actual label names

      predicted_label = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

      predicted_labels2=labletoid[predicted_label[0]]

      predicted_labels.append(predicted_labels2)
      true_labels.append(labels2)

    except ValueError as ve:
      print("ve")

    except IndexError as ie:
      predicted_labels.append(1)
      true_labels.append(labels2)


  return predicted_labels,true_labels

ppredicted_labels,true_labels = pred_pre(root=test_paths,processor=processor)

In [ ]:
print(len(ppredicted_labels))
print(len(true_labels))

confusion metrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming 'true_labels' are the true labels and 'predicted_labels' are the predicted labels
#true_labels = [0, 1, 0, 1, 1, 0, 0, 1, 1, 1]
true_labels=true_labels
#predicted_labels = [0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
predicted_labels=ppredicted_labels

# Define class labels
class_names = ['Action', 'Drugs', 'Gambling', 'LSD', 'Weapon']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=class_names, yticklabels=class_names)

# Add labels, title, and colorbar
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.yticks(rotation=0)
plt.xticks(rotation=45)
plt.show()


Accuracy report

In [ ]:
#accuracy report
from sklearn.metrics import accuracy_score, classification_report

# Calculate the accuracy score
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# Generate a classification report
report = classification_report(true_labels, predicted_labels)
print("Classification Report:")
print(report)